# Visualize Supervisor Graph

**Kernel:** Python (DV360 Supervisor)  
**First, load environment variables from .env file**


In [3]:
# Load environment variables from .env file FIRST
import os
from pathlib import Path
from dotenv import load_dotenv

# Find .env file (could be in backend/ or project root)
env_paths = [
    Path.cwd() / ".env",  # Current directory
    Path.cwd().parent / ".env",  # Parent directory (project root)
]

for env_path in env_paths:
    if env_path.exists():
        load_dotenv(env_path)
        print(f"✓ Loaded .env from: {env_path}")
        break
else:
    print("⚠ No .env file found. Make sure ANTHROPIC_API_KEY or OPENAI_API_KEY is set in environment.")

# Check if API key is loaded
if os.getenv("ANTHROPIC_API_KEY"):
    print(f"✓ ANTHROPIC_API_KEY found")
elif os.getenv("OPENAI_API_KEY"):
    print(f"✓ OPENAI_API_KEY found")
else:
    print("⚠ No API key found! Set ANTHROPIC_API_KEY or OPENAI_API_KEY in .env file")


✓ Loaded .env from: /Users/neilblyth/Documents/Apps/LangraphDV360/.env
✓ ANTHROPIC_API_KEY found


# Import supervisor graph
import sys
from pathlib import Path

# Add backend/src to path
backend_src = Path.cwd() / "src"
if str(backend_src) not in sys.path:
    sys.path.insert(0, str(backend_src))

from agents.supervisor import super_graph

# Visualize the graph
print("Supervisor Graph Structure:")
print("=" * 50)

# Get graph representation
graph_mermaid = super_graph.get_graph().draw_mermaid()
print("\nMermaid Diagram:")
print("-" * 50)
print(graph_mermaid)

# Display as image if IPython is available
try:
    from IPython.display import Image, display
    import subprocess
    import tempfile
    
    # Create temporary file for mermaid
    with tempfile.NamedTemporaryFile(mode='w', suffix='.mmd', delete=False) as f:
        f.write(graph_mermaid)
        mermaid_file = f.name
    
    # Try to render using mermaid CLI or online service
    print("\n" + "=" * 50)
    print("Graph Visualization:")
    print("-" * 50)
    
    # For now, just show the mermaid code
    # You can copy this to https://mermaid.live/ to visualize
    print("\nCopy the Mermaid diagram above to https://mermaid.live/ to visualize")
    
except Exception as e:
    print(f"\nCould not render image: {e}")
    print("Copy the Mermaid diagram above to https://mermaid.live/ to visualize")


In [4]:
# Alternative: Print graph structure as text
print("Graph Nodes and Edges:")
print("=" * 50)

graph = super_graph.get_graph()
print(f"\nNodes: {list(graph.nodes.keys())}")
print(f"\nEdges:")
for edge in graph.edges:
    print(f"  {edge.source} -> {edge.target}")

# Show node details
print(f"\nNode Details:")
for node_name, node in graph.nodes.items():
    print(f"\n  {node_name}:")
    print(f"    Type: {type(node).__name__}")


Graph Nodes and Edges:


NameError: name 'super_graph' is not defined

In [5]:
# Simple text visualization
print("Graph Flow:")
print("=" * 50)
print("""
START
  |
  v
supervisor (routes to budget/performance/FINISH)
  |
  +---> budget (queries Snowflake budget data)
  |       |
  |       v
  |     supervisor (back to supervisor)
  |
  +---> performance (queries Snowflake performance data)
  |       |
  |       v
  |     supervisor (back to supervisor)
  |
  +---> FINISH (END)
  
Flow: supervisor -> budget/performance -> supervisor -> ... -> FINISH
""")


Graph Flow:

START
  |
  v
supervisor (routes to budget/performance/FINISH)
  |
  +---> budget (queries Snowflake budget data)
  |       |
  |       v
  |     supervisor (back to supervisor)
  |
  +---> performance (queries Snowflake performance data)
  |       |
  |       v
  |     supervisor (back to supervisor)
  |
  +---> FINISH (END)

Flow: supervisor -> budget/performance -> supervisor -> ... -> FINISH



In [6]:
from langchain_core.messages import HumanMessage
from src.agents.supervisor import super_graph

# Test the graph
result = await super_graph.ainvoke({
    "messages": [HumanMessage(content="What is my budget?")],
    "next": None,
    "session_id": None,
    "user_id": "test"
})

print(result["messages"][-1].content)


ValidationError: 3 validation errors for Settings
api_reload
  Input should be a valid boolean, unable to interpret input [type=bool_parsing, input_value='true  # Set to false in production', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/bool_parsing
vector_dimension
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='1536  # For OpenAI text-embedding-3-small', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing
memory_top_k
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='5  # Number of relevant memories to retrieve', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing